In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from pathlib import Path
import os
import datetime

In [2]:
# get path to cwd and set project root
notebook_dir = Path.cwd()
project_root = notebook_dir.parent

# define full path to dataset and load
data_path =  os.path.join(project_root, 'data/gedi_waveforms_tf.npz')
data = np.load(data_path)

In [3]:
# Extract waveform data
waveforms = data['waveforms']

# Add new axis to waveform data
waveforms = waveforms[..., np.newaxis]

# inspect waveform data and shape
print(waveforms.shape)
print(waveforms)

(10546, 500, 1)
[[[-0.92182818]
  [-1.11135732]
  [-1.0882749 ]
  ...
  [-0.82720233]
  [-0.7545843 ]
  [-0.65852474]]

 [[-0.51685445]
  [-0.91077666]
  [-1.0163088 ]
  ...
  [-1.14346151]
  [-0.74755905]
  [-0.30184295]]

 [[-0.47643436]
  [-0.54564899]
  [-0.33103624]
  ...
  [ 0.23716828]
  [ 0.30925776]
  [ 0.10473613]]

 ...

 [[-1.01867713]
  [-1.48746914]
  [-1.54916126]
  ...
  [-0.18743011]
  [-0.19228905]
  [-0.06010556]]

 [[ 0.06117187]
  [ 0.00706989]
  [-0.20781391]
  ...
  [-0.20177718]
  [-0.17382068]
  [-0.17141481]]

 [[-0.78835739]
  [-0.72094595]
  [-0.28164888]
  ...
  [ 0.97619698]
  [ 0.64958933]
  [ 0.17167537]]]


In [4]:
# Split dataset into training and validation sets (80/20 split)
x_train, x_temp = train_test_split(waveforms, test_size = 0.3, random_state = 0)
x_test, x_val = train_test_split(x_temp, test_size = 0.5, random_state = 0)

# inspect the shape of the training and validation sets
print(f"Training data:  {x_train.shape}")
print(f"Testing data:  {x_test.shape}")
print(f"Validation data: {x_val.shape}")

Training data:  (7382, 500, 1)
Testing data:  (1582, 500, 1)
Validation data: (1582, 500, 1)


In [5]:
from tensorflow.keras import layers, models, optimizers
import numpy as np

def build_autoencoder(input_shape=(500, 1), latent_shape=(4, 4),
                                        dropout_rate=0.0, use_batchnorm=False, use_mlp_bottleneck=False):
    """
    Build a convolutional autoencoder with structured latent space (latent_len, latent_dim),
    using your proven dense bottleneck strategy.
    """

    latent_len, latent_dim = latent_shape
    latent_size = latent_len * latent_dim

    # Encoder
    inputs = layers.Input(shape=input_shape, name='input_layer')

    x = layers.Conv1D(32, 3, padding='same')(inputs)
    if use_batchnorm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(2, padding='same')(x)

    x = layers.Conv1D(64, 3, padding='same')(x)
    if use_batchnorm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(2, padding='same')(x)  # Output: (125, 64)

    x = layers.Flatten()(x)  # shape: (125 * 64 = 8000)

    if dropout_rate > 0:
        x = layers.Dropout(dropout_rate)(x)

    # Bottleneck
    if use_mlp_bottleneck:
        x = layers.Dense(128, activation='relu')(x)
    bottleneck = layers.Dense(latent_size, activation='linear', name='bottleneck')(x)
    reshaped_bottleneck = layers.Reshape((latent_len, latent_dim), name='latent_reshape')(bottleneck)

    encoder = models.Model(inputs, reshaped_bottleneck, name='encoder')

    # Decoder
    decoder_input = layers.Input(shape=(latent_len, latent_dim), name='decoder_input')
    x = layers.Flatten()(decoder_input)  # shape: (latent_len * latent_dim,)
    x = layers.Dense(125 * 64, activation='relu')(x)
    x = layers.Reshape((125, 64))(x)

    x = layers.Conv1D(64, 3, padding='same')(x)
    if use_batchnorm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.UpSampling1D(2)(x)

    x = layers.Conv1D(32, 3, padding='same')(x)
    if use_batchnorm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.UpSampling1D(2)(x)

    decoded = layers.Conv1D(1, 3, padding='same', activation='linear')(x)

    decoder = models.Model(decoder_input, decoded, name='decoder')

    # Autoencoder
    autoencoder_output = decoder(encoder(inputs))
    autoencoder = models.Model(inputs, autoencoder_output, name='autoencoder')
    autoencoder.compile(optimizer=optimizers.Adam(1e-3), loss='mse')

    return autoencoder, encoder, decoder


In [6]:
def save_reconstruction_plot(model, data, experiment_id, test_loss, config, n=10, save_dir=None, seed=42):
    if save_dir is None:
        save_dir = os.path.join(project_root, 'plots')
    os.makedirs(save_dir, exist_ok=True)

    index_path = os.path.join(save_dir, "selected_indices.npy")

    # Load or generate consistent indices
    if os.path.exists(index_path):
        indices = np.load(index_path)
    else:
        np.random.seed(seed)
        indices = np.random.choice(len(data), size=n, replace=False)
        np.save(index_path, indices)

    selected_data = data[indices]
    reconstructions = model.predict(selected_data, verbose=0)

    plt.figure(figsize=(12, 3 * n))

    config_str = (
        f"Latent Shape: {config['latent_shape']} | Test MSE: {test_loss:.4f}"
    )
    plt.suptitle(f"{experiment_id} — {config_str}", fontsize=12, y=1.02)

    for i in range(n):
        plt.subplot(n, 2, 2*i + 1)
        plt.plot(selected_data[i].squeeze(), color='blue')
        plt.title(f"Original #{indices[i]}")
        plt.grid(True)

        plt.subplot(n, 2, 2*i + 2)
        plt.plot(reconstructions[i].squeeze(), color='orange')
        plt.title(f"Reconstructed #{indices[i]}")
        plt.grid(True)

    plt.tight_layout()
    save_path = os.path.join(save_dir, f"{experiment_id}_reconstruction.png")
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"Saved reconstruction plot to: {save_path}")

In [7]:
latent_configs = [
    {"latent_shape": (4, 16)},
    {"latent_shape": (2, 4)},
    {"latent_shape": (4, 4)},
    {"latent_shape": (8, 8)},
    {"latent_shape": (4, 8)},
    {"latent_shape": (2, 8)},
    {"latent_shape": (1, 8)}  # baseline
]

In [8]:
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime


# Experiment tracking
results_log = []

def run_latent_experiment(
    experiment_id,
    latent_shape,
    input_shape=(500, 1),
    learning_rate=1e-3,
    dropout_rate=0.0,
    batch_size=64,
    epochs=20,
    save_models=True,
    results_path=os.path.join(project_root, "models/latent_shape_experiment_results.csv")
):
    print(f"\nRunning {experiment_id} | Latent Shape: {latent_shape}, LR: {learning_rate}, Dropout: {dropout_rate}")

    # Build model
    autoencoder, encoder, decoder = build_autoencoder(
        input_shape=input_shape,
        latent_shape=latent_shape,
        dropout_rate=dropout_rate,
        use_batchnorm = False,
        use_mlp_bottleneck = False
    )

    autoencoder.compile(optimizer=optimizers.Adam(learning_rate), loss='mse')

    # Set up TensorBoard
    log_dir = os.path.join(project_root, f"logs/{experiment_id}_{datetime.now().strftime('%Y%m%d-%H%M%S')}")
    tensorboard_cb = TensorBoard(log_dir=log_dir)

    # Train model
    history = autoencoder.fit(
        x_train, x_train,
        validation_data=(x_test, x_test),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[tensorboard_cb]
    )

    # Evaluate
    train_loss = history.history['loss'][-1]
    val_loss = history.history['val_loss'][-1]
    test_loss = autoencoder.evaluate(x_test, x_test, verbose=0)

    # Save model + embeddings (optional)
    if save_models:
        model_dir = os.path.join(project_root, "models")
        os.makedirs(model_dir, exist_ok=True)
        autoencoder.save(os.path.join(model_dir, f"autoencoder_{experiment_id}.keras"))
        encoder.save(os.path.join(model_dir, f"encoder_{experiment_id}.keras"))
        embeddings = encoder.predict(waveforms, verbose=0)
        np.save(os.path.join(model_dir, f"embeddings_{experiment_id}.npy"), embeddings)

    # Save reconstruction plot
    save_reconstruction_plot(
        model=autoencoder,
        data=x_test,
        experiment_id=experiment_id,
        test_loss=test_loss,
        config={'latent_shape': latent_shape},
        n=10,
        save_dir=os.path.join(project_root, "plots")
    )

    # Log results
    result = {
        'Experiment': experiment_id,
        'Latent Shape': str(latent_shape),
        'LR': learning_rate,
        'Dropout': dropout_rate,
        'Epochs': epochs,
        'Train Loss': train_loss,
        'Val Loss': val_loss,
        'Test Loss': test_loss
    }
    results_log.append(result)

    df_results = pd.DataFrame(results_log)
    df_results.to_csv(results_path, index=False)
    print(f"{experiment_id} complete — Test MSE: {test_loss:.4f} — Logged to {results_path}")

In [9]:
start_idx = 1 
for i, config in enumerate(latent_configs):
    latent_shape = config['latent_shape']
    experiment_id = f"exp_{(start_idx + i):02d}_{latent_shape[0]}x{latent_shape[1]}"
    
    run_latent_experiment(
        experiment_id=experiment_id,
        latent_shape=latent_shape,
        input_shape=(500, 1),
        learning_rate=1e-3,
        dropout_rate=0.0,
        batch_size=64,
        epochs=30,
        save_models=True,
        results_path=os.path.join(project_root, "models/latent_shape_experiment_results.csv")
    )


Running exp_01_4x16 | Latent Shape: (4, 16), LR: 0.001, Dropout: 0.0
Saved reconstruction plot to: c:\Users\Zachary\Downloads\gedi_waveform_processor_library\plots\exp_01_4x16_reconstruction.png
exp_01_4x16 complete — Test MSE: 0.0164 — Logged to c:\Users\Zachary\Downloads\gedi_waveform_processor_library\models/latent_shape_experiment_results.csv

Running exp_02_2x4 | Latent Shape: (2, 4), LR: 0.001, Dropout: 0.0
Saved reconstruction plot to: c:\Users\Zachary\Downloads\gedi_waveform_processor_library\plots\exp_02_2x4_reconstruction.png
exp_02_2x4 complete — Test MSE: 0.0458 — Logged to c:\Users\Zachary\Downloads\gedi_waveform_processor_library\models/latent_shape_experiment_results.csv

Running exp_03_4x4 | Latent Shape: (4, 4), LR: 0.001, Dropout: 0.0
Saved reconstruction plot to: c:\Users\Zachary\Downloads\gedi_waveform_processor_library\plots\exp_03_4x4_reconstruction.png
exp_03_4x4 complete — Test MSE: 0.0331 — Logged to c:\Users\Zachary\Downloads\gedi_waveform_processor_library\m

In [10]:
# run_latent_experiment(
#     experiment_id="latent_10x8",
#     latent_shape=(10, 8),
#     input_shape=(500, 1),
#     learning_rate=1e-3,
#     dropout_rate=0.0,
#     batch_size=64,
#     epochs=30
# )